In [1]:
# Required libraries
from pathlib import Path
import sys
import os 
import pandas as pd
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
import datetime
import zipfile 
from timeit import default_timer as timer
# Paths
sys.path.append(os.path.join(Path(os.getcwd()).parent))  
data_path = os.path.join(os.path.join(Path(os.getcwd()).parent), 'data')
data_per_day_path = os.path.join(os.path.join(Path(os.getcwd()).parent), 'data','data_per_day')

In [2]:
# get the csv file now
csv_files = [f for f in os.listdir(data_per_day_path) if os.path.isfile(os.path.join(data_per_day_path, f))]
csv_files = [file for file in csv_files if '.csv' in file]
csv_files = [os.path.join(data_per_day_path, x) for x in csv_files]
csv_file = csv_files[0]
csv_file

'/home/patricklucescu/Dropbox/Projects/financial_volatility/financial_volatility/data/data_per_day/SPY_20200203.csv'

In [3]:
%%time
data_df = pd.read_csv(csv_file)
data_df.DT = pd.to_datetime(data_df.DT)

# non zero quotes
data_df = data_df.loc[(data_df.BID>0) & (data_df.BIDSIZ>0) & (data_df.ASK>0) & (data_df.ASKSIZ>0)]

# autoselect exchange
data_df['total_size'] = data_df.BID + data_df.ASK
most_volume = data_df.groupby(['EX']).sum().total_size.copy()
desired_exchange = most_volume.idxmax()
data_df = data_df.loc[data_df.EX == desired_exchange]

# delete negative spreads
data_df = data_df.loc[data_df.ASK > data_df.BID]

# mergeQuotesSameTimestamp
ex = data_df.EX.values[0]
sym_root = data_df.SYM_ROOT.values[0]
data_df.drop(columns=['SYM_SUFFIX', 'total_size'], inplace=True)
data_df = data_df.groupby(['DT']).median()
data_df['EX'] = ex
data_df['SYM_ROOT'] = sym_root
data_df.reset_index(drop=False, inplace=True)

# remove entries with spread > 50 * daily medial 
data_df['SPREAD'] = data_df.ASK - data_df.BID
data_df = data_df.loc[data_df['SPREAD'] < 50 * data_df['SPREAD'].median()]

# remove outliers using the centered rolling window approach 
def compute_diff(x):
    return x.values[window] - np.median(np.delete(x.values,window))

window = 25
data_df.sort_values(by=['DT'], inplace=True)
data_df['SPREAD_DIFF'] = data_df.SPREAD.rolling(2*window+1, min_periods=2*window+1, center=True).apply(compute_diff)
data_df = data_df.dropna()
data_df = data_df.loc[data_df['SPREAD_DIFF'] < 10 * data_df['SPREAD_DIFF'].mean()]
data_df = data_df.reset_index(drop=True)

CPU times: user 1min 15s, sys: 801 ms, total: 1min 15s
Wall time: 1min 15s
